In [11]:
from collections import Counter
lst = ["aa", "bb", "aa", "cc"]
count = Counter(lst)
# print(count)

lst = [["aa", "bb", "aa", "cc"], ["aa", "bb", "aa", "cc"], ["aa", "bb", "aa", "cc"]]
c = Counter([_ for i in range(len(lst)) for _ in lst[i]])
print(c)

Counter({'aa': 6, 'bb': 3, 'cc': 3})


# train & test data creation

In [7]:
import json

with open("./tqa_train_val_test/train/tqa_v1_train.json") as f:
# with open("./tqa_train_val_test/test/tqa_v2_test.json") as f:
    train = json.load(f)

# trainはリストで、各要素に質問と回答の候補、答えが辞書型のデータとして格納されています。
# train[0] = {'answerChoices': {'a': 'solid Earth.',
#  'b': 'Earths oceans.',
#  'c': 'Earths atmosphere.',
#  'd': 'all of the above'},
# 'correctAnswer': 'd',
# 'question': 'Earth science is the study of'}
# fixed_data = train[0]
fixed_data = []
for t in train:
    for key, value in t["questions"]["nonDiagramQuestions"].items():
        if value["beingAsked"] is not None \
                and value["answerChoices"] is not None \
                and value["correctAnswer"] is not None \
                and value["questionType"] == "Multiple Choice":
            dic = {
                "question": value["beingAsked"]["processedText"],
                "answerChoices": {
                    "a": value["answerChoices"].get("a")["processedText"] if value["answerChoices"].get("a") is not None else "",
                    "b": value["answerChoices"].get("b")["processedText"] if value["answerChoices"].get("b") is not None else "",
                    "c": value["answerChoices"].get("c")["processedText"] if value["answerChoices"].get("c") is not None else "",
                    "d": value["answerChoices"].get("d")["processedText"] if value["answerChoices"].get("d") is not None else "",
                },
                "correctAnswer": value["correctAnswer"]["processedText"],
            }
            fixed_data.append(dic)
print(fixed_data[0])

with open("./tqa_train_val_test/train.json", "w") as j:
# with open("./tqa_train_val_test/val.json", "w") as j:
    j.write(json.dumps(fixed_data))


{'question': 'Earth science is the study of', 'answerChoices': {'a': 'solid Earth.', 'b': 'Earths oceans.', 'c': 'Earths atmosphere.', 'd': 'all of the above'}, 'correctAnswer': 'd'}


# 単語辞書の作成

In [ ]:
import json
import re
from collections import Counter
import itertools

with open("./tqa_train_val_test/train.json") as f:
    j = json.load(f)

wakatiO = []
for row in j:
    for key, value in row.items():
        # keys: question, choices, correctAnswer
        if key == "question" or key == "correctAnswer":
            wakatiO.append(value.lower().split())
        else:
            for k, v in value.items():
                # print(v)
                re.sub("[_.,]+", "", v)
                wakatiO.append(v.lower().split())

# 単語の出現数をカウントしてください
# word_freq = Counter([_ for i in range(len(wakatiO)) for _ in wakatiO[i]])
word_freq = Counter(itertools.chain(*wakatiO))

# 単語を出現数順に並べ替え、dicに追加してください
dic = []
for word_uniq in word_freq.most_common():
    dic.append(word_uniq[0])

# 単語にIDを付与し辞書を作成
# enumerate(dic) は、for文を使用して`dic`からインデックスを`i`、要素を`word_uniq`に取得
dic_inv = {}
for i, word_uniq in enumerate(dic, start=1):
    dic_inv.update({word_uniq: i})

# 辞書の表示
print(dic_inv)
with open("./tqa_train_val_test/word2id.json", "w") as j:
    j.write(json.dumps(dic_inv))